In [ ]:
!pip install tweepy pandas nltk sqlalchemy psycopg2-binary flask schedule

In [ ]:
# Configuration setup
from configparser import ConfigParser

config = ConfigParser()

config['twitter'] = {
    'consumer_key': '',
    'consumer_secret': '',
    'access_token': '',
    'access_token_secret': ''
}

with open('/content/config.ini', 'w') as configfile:
    config.write(configfile)

print("Configuration file created.")

Configuration file created.


In [ ]:
#libraries
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from collections import Counter
import re
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from flask import Flask, jsonify
import configparser
import schedule
import time
from datetime import datetime, timedelta
import random

#NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

config = configparser.ConfigParser()

config['database'] = {
    'connection_string': ''
}

with open('/content/config.ini', 'w') as configfile:
    config.write(configfile)

config.read('/content/config.ini')

Base = declarative_base()

class FashionTweet(Base):
    __tablename__ = 'fashion_tweets'

    id = Column(Integer, primary_key=True)
    text = Column(String)
    clean_text = Column(String)
    user = Column(String)
    created_at = Column(DateTime)
    hashtags = Column(String)
    sentiment = Column(Float)

engine = create_engine(config['database']['connection_string'])
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

fashion_keywords = ['fashion', 'style', 'trend', 'outfit', 'clothes', 'wear', 'dress', 'shoes', 'accessories',
                    'designer', 'collection', 'runway', 'model', 'brand', 'luxury', 'vintage', 'sustainable',
                    'couture', 'vogue', 'chic', 'elegant', 'glamour', 'fashionweek', 'streetwear']
fashion_influencers = ['vogue', 'elle', 'harpersbazaar', 'wmag', 'instyle']

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^\w\s#]', '', text)
    return text.lower()

def extract_hashtags(text):
    return [word for word in text.split() if word.startswith('#')]

def analyze_sentiment(text):
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)['compound']

def generate_simulated_tweet():
    text = f"Check out this {random.choice(fashion_keywords)} {random.choice(fashion_keywords)}! #{random.choice(fashion_keywords)} #{random.choice(fashion_keywords)}"
    clean_text_content = clean_text(text)
    hashtags = extract_hashtags(clean_text_content)
    sentiment = analyze_sentiment(clean_text_content)

    return FashionTweet(
        text=text,
        clean_text=clean_text_content,
        user=random.choice(fashion_influencers),
        created_at=datetime.now() - timedelta(minutes=random.randint(0, 60)),
        hashtags=','.join(hashtags),
        sentiment=sentiment
    )

def collect_simulated_tweets():
    session = Session()
    for _ in range(200): #200 tweets at a time
        tweet = generate_simulated_tweet()
        session.add(tweet)

    session.commit()
    session.close()
    print("Simulated tweets collected and stored.")

def analyze_trends():
    session = Session()
    tweets = session.query(FashionTweet).order_by(FashionTweet.created_at.desc()).limit(1000).all()

    word_freq = Counter()
    hashtag_freq = Counter()

    for tweet in tweets:
        words = word_tokenize(tweet.clean_text)
        word_freq.update([w for w in words if w in fashion_keywords])
        hashtag_freq.update(tweet.hashtags.split(','))

    top_words = word_freq.most_common(10)
    top_hashtags = hashtag_freq.most_common(10)

    sentiment_dist = pd.Series([tweet.sentiment for tweet in tweets]).apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral')).value_counts(normalize=True)

    session.close()

    return {
        'top_words': top_words,
        'top_hashtags': top_hashtags,
        'sentiment_distribution': sentiment_dist.to_dict()
    }

app = Flask(__name__) #HTTP server to call this model and use it with any required provider or web frontend

@app.route('/generate_report', methods=['GET'])
def generate_report():
    trends = analyze_trends()
    return jsonify(trends)

def run_collection():
    while True:
        collect_simulated_tweets()
        print("Simulated tweets collected and stored.")
        time.sleep(60)

if __name__ == '__main__':
    collect_simulated_tweets()

    report = analyze_trends()
    print("Fashion Trend Report:")
    print("Top Words:", report['top_words'])
    print("Top Hashtags:", report['top_hashtags'])
    print("Sentiment Distribution:", report['sentiment_distribution'])
    # run_collection()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
<ipython-input-4-f9ba7e3353a3>:34: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Simulated tweets collected and stored.
Fashion Trend Report:
Top Words: [('brand', 78), ('designer', 75), ('glamour', 75), ('style', 73), ('vogue', 73), ('outfit', 72), ('runway', 71), ('luxury', 69), ('couture', 69), ('accessories', 68)]
Top Hashtags: [('#style', 43), ('#luxury', 42), ('#vogue', 41), ('#outfit', 40), ('#glamour', 38), ('#brand', 38), ('#clothes', 37), ('#couture', 36), ('#runway', 35), ('#chic', 35)]
Sentiment Distribution: {'neutral': 0.7575, 'positive': 0.2425}
